In [37]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)
logger = logging.getLogger(__name__)

In [10]:
from evaluation_utils import set_env_var

set_env_var("MEMORY_BACKEND", "mem0")

In [ ]:
!docker-compose stop locomo memory responder coordinator qdrant neo4j
!docker-compose rm -f locomo memory responder coordinator qdrant neo4j
!docker volume rm dmas-long-context-memory_locomo-data dmas-long-context-memory_qdrant-data dmas-long-context-memory_neo4j-data dmas-long-context-memory_neo4j-logs

In [ ]:
!docker-compose up --build -d
# 2m 23s

#0 building with "desktop-linux" instance using docker driver

#1 [locomo internal] load build definition from Dockerfile
#1 transferring dockerfile: 294B 0.0s done
#1 DONE 0.0s

#2 [responder internal] load build definition from Dockerfile
#2 transferring dockerfile: 294B 0.0s done
#2 DONE 0.1s

#3 [memory internal] load build definition from Dockerfile
#3 transferring dockerfile: 294B 0.0s done
#3 DONE 0.1s

#4 [memory internal] load metadata for docker.io/library/python:3.11-slim
#4 DONE 1.1s

#5 [locomo internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.1s

#6 [responder internal] load .dockerignore
#6 transferring context: 2B 0.0s done
#6 DONE 0.1s

#7 [memory internal] load .dockerignore
#7 transferring context: 2B done
#7 DONE 0.1s

#8 [memory 1/6] FROM docker.io/library/python:3.11-slim@sha256:9353e32cde8a32ebf11bfd37159b8307f2366ebe536578b8eaa2a6065f04c3c9
#8 DONE 0.0s

#9 [responder internal] load build context
#9 transferring context: 508B done
#9 D

 ollama Pulling 
 4b3ffd8ccb52 Already exists 
 98bf6a5ec929 Already exists 
 773786a5e0f4 Already exists 
 027d4f2abd02 Pulling fs layer 
 027d4f2abd02 Downloading [===========================>                       ]   1.05GB/1.877GB
 027d4f2abd02 Downloading [===========================>                       ]  1.051GB/1.877GB
 027d4f2abd02 Downloading [============================>                      ]  1.052GB/1.877GB
 027d4f2abd02 Downloading [============================>                      ]  1.053GB/1.877GB
 027d4f2abd02 Downloading [============================>                      ]  1.054GB/1.877GB
 027d4f2abd02 Downloading [============================>                      ]  1.055GB/1.877GB
 027d4f2abd02 Downloading [============================>                      ]  1.056GB/1.877GB
 027d4f2abd02 Downloading [============================>                      ]  1.057GB/1.877GB
 027d4f2abd02 Downloading [============================>                      ]  1.05

In [19]:
conversation_id = 0

In [71]:
import requests

response = requests.post(f"http://localhost:8003/conversation/load/{conversation_id}/session/1")

logger.info(f"Status: {response.status_code}")
logger.info(f"Response: {response.json()}")
# 54.1s

2025-11-04 18:37:15,342 - INFO - Status: 200
2025-11-04 18:37:15,353 - INFO - Response: {'status': 'success', 'conversation_id': 0, 'added': 18, 'skipped': 0, 'failed': 0, 'total_sessions': 1, 'total_processed': 18, 'failures': None, 'results': [{'session': 'session_1', 'session_index': 1, 'turn_index': 0, 'text_snippet': 'Hey Mel! Good to see you! How have you been?', 'speaker': 'caroline', 'timestamp': '2023-05-08T13:56:00', 'memory_result': {'results': []}}, {'session': 'session_1', 'session_index': 1, 'turn_index': 1, 'text_snippet': "Hey Caroline! Good to see you! I'm swamped with the kids & work. What's up with you? Anything new?", 'speaker': 'melanie', 'timestamp': '2023-05-08T13:56:00', 'memory_result': {'results': []}}, {'session': 'session_1', 'session_index': 1, 'turn_index': 2, 'text_snippet': 'I went to a LGBTQ support group yesterday and it was so powerful.', 'speaker': 'caroline', 'timestamp': '2023-05-08T13:56:00', 'memory_result': {'results': []}}, {'session': 'session

In [ ]:
import requests

response = requests.get(f"http://localhost:8002/conversations/index/{conversation_id}/questions")

logger.info(f"Status: {response.status_code}")
questions = response.json().get("questions", [])
logger.info(questions)

Status: 200
[{'sample_id': 'conv-26', 'question_id': 'conv-26_q_0', 'question': 'When did Caroline go to the LGBTQ support group?', 'answer': '7 May 2023', 'category': 2, 'evidence': ['D1:3']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_1', 'question': 'When did Melanie paint a sunrise?', 'answer': '2022', 'category': 2, 'evidence': ['D1:12']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_2', 'question': 'What fields would Caroline be likely to pursue in her educaton?', 'answer': 'Psychology, counseling certification', 'category': 3, 'evidence': ['D1:9', 'D1:11']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_3', 'question': 'What did Caroline research?', 'answer': 'Adoption agencies', 'category': 1, 'evidence': ['D2:8']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_4', 'question': "What is Caroline's identity?", 'answer': 'Transgender woman', 'category': 1, 'evidence': ['D1:5']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_5', 'question': 'When did Me

In [70]:
for i, question in enumerate(questions):
    try:
        response = requests.post(
            "http://localhost:8003/ask",
            params={"question": question.get("question")}
        )
        response.raise_for_status()
        
        logger.info(f"Question {i+1}: {question.get("question")}")
        answer = response.json().get("answer")
        logger.info(f"Answer: {answer}\n")
        
    except Exception as e:
        logger.info(f"Error on question {i+1}: {e}\n")
    
    break
# 10.1s

2025-11-04 18:15:14,909 - INFO - Question 1: When did Caroline go to the LGBTQ support group?
2025-11-04 18:15:14,911 - INFO - Answer: answer_question "When did Caroline attend the LGBTQ support group?"

